In [21]:
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import scipy as sc
import csv
import plotly as py
import googlemaps as goog
import geopy as geo
import sklearn as skl
from sklearn import cluster

# import data table as a data frame
#MedicareTable = pd.read_table('D:\Medicare_Aggregate\Medicare_Physician_and_Other_Supplier_NPI_Aggregate_CY2014.txt', sep='\t',engine='python')

In [13]:
#Display 100 rows from columns
#MedicareTable.loc[1:20,:]

#pd.describe(MedicareTable.loc[:, 'average_Medicare_standard_amt'])
#MedicareTable['total_drug_unique_benes'].dropna().describe()

In [ ]:
#submittedCharge = MedicareTable[MedicareTable['nppes_entity_code']=='I'].loc[:,'total_submitted_chrg_amt']
#medicarePayment = MedicareTable[MedicareTable['nppes_entity_code']=='I'].loc[:,'total_medicare_payment_amt']
#
#plt.scatter(submittedCharge, (medicarePayment/submittedCharge))
#plt.show()

In [7]:
# Create new matrix, selectedVarMatrix, containing only our three variables to cluster on
selectedVar = ['total_services','total_submitted_chrg_amt','total_medicare_stnd_amt']
reducedVarMatrix = MedicareTable[MedicareTable['nppes_entity_code']=='I'].loc[:,selectedVar]
#figure = pd.tools.plotting.scatter_matrix(selectedVarMatrix, alpha=0.2, figsize=(15,15))
#plt.show()


In [5]:
# Create reducedVarMatrix, which eliminates any tuples containing 
#reducedVarMatrix = selectedVarMatrix[pd.notnull(MedicareTable['total_services'])].loc[:,:]
#reducedVarMatrix = reducedVarMatrix[pd.notnull(reducedVarMatrix['total_submitted_chrg_amt'])].loc[:,:]
#reducedVarMatrix = reducedVarMatrix[pd.notnull(reducedVarMatrix['total_medicare_stnd_amt'])].loc[:,:]
#reducedVarMatrix = selectedVarMatrix

In [8]:
# Normalize variables
normVarMatrix = skl.preprocessing.normalize(reducedVarMatrix, norm='l1', axis=0)

scale = skl.preprocessing.StandardScaler().fit(reducedVarMatrix[:])
normVarMatrix = scale.transform(reducedVarMatrix[:])

print("Column one mean is now: ", np.mean(normVarMatrix[:,0]))
print("Column two standard deviation is now: ", np.std(normVarMatrix[:,1]))

Column one mean is now:  4.58262581174e-17
Column two standard deviation is now:  1.0


In [10]:
#Split data into 5 random samples
#randomSampleLength = len(normVarMatrix[:,0])
#random = np.random.uniform(low = 0.0, high=5.0, size=randomSampleLength)

#random1 = all(0.0 <= random < 1)
#sample1 = normVarMatrix[(0.0 <= random < 1), :]
#random >= 0

#sample1 = normVarMatrix[0:50000,:]

In [87]:
kmeans = cluster.KMeans(n_clusters = 5)
kmeans.fit(normVarMatrix)

centroid = kmeans.cluster_centers_
labels = kmeans.labels_

print(centroid)
print(labels)

[[ -0.11784958  -0.2562531   -0.24235344]
 [ 27.68840978  16.01640696  17.64281585]
 [  1.49261886   3.45449123   3.23366325]
 [  0.19932412   0.75112611   0.68047754]
 [  7.69666604   8.88652929   9.44396467]]
[3 3 0 ..., 0 0 0]


In [ ]:
color_map = {0:'r',
            1:'k',
            2:'b',
            3:'r',
            4:'c',
            5:'m',
            6:'y',
            7:'w',
            8:'b',
            9:'y', }

label_color = [color_map[l] for l in labels]

plt.scatter(reducedVarMatrix.loc[:,'total_submitted_chrg_amt'],reducedVarMatrix.loc[:,'total_medicare_stnd_amt'],c=label_color)
plt.show()

In [43]:
#figure = pd.tools.plotting.scatter_matrix(selectedVarMatrix, alpha=0.2, figsize=(15,15))
#plt.show()

In [ ]:
outliers=MedicareTable[labels==1].loc[:,'nppes_provider_zip']
outliers=outliers[0:300,:]

In [ ]:
newZips = np.arange(len(outliers))

for i in range(len(outliers)):
    if int(outliers[i]) > 99999:
        remainder = int(outliers[i]) % 10000
        newZips[i] = int(outliers[i]) - remainder
        newZips[i] = int(outliers[i])/10000
    else:
        remainder = int(outliers[i]) % 10000000
        newZips[i] = int(outliers[i]) - remainder
        newZips[i] = int(outliers[i])/10000

In [72]:
for i in range(len(newZips)):
    if newZips[i] < 10000:
        np.delete(newZips, i)

In [73]:
longitudes = np.arange(len(outliers))
latitudes = np.arange(len(outliers))
geolocator = geo.geocoders.Nominatim()

for i in range(len(outliers)):
    zipcode = str(newZips[i])
    USA = ", United States"
    search = zipcode + USA
    location = geolocator.geocode(newZips[i])
    if ((location is None) == False):
        latitudes[i] = location.latitude
        longitudes[i] = location.longitude

States Road, St Anne's on the Sea, Lancs, North West England, England, UK


In [ ]:
coordinates = [longitudes, latitudes]

kmeans = cluster.KMeans(n_clusters = 4)
kmeans.fit(coordinates)

centroid = kmeans.cluster_centers_
labels = kmeans.labels_

print(centroid)
print(labels)

In [ ]:
plt.scatter(longitudes,latitudes,c=label_color)
plt.show()